<!-- Autoheader begin -->
<hr/>
<div id="navtitle_3_2_py" style="text-align:center; font-size:16px">III.2 Optimal Control for STIRAP</div>
<hr/>
<table style="width: 100%">
  <tr>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
        <a href="py_exercise_3_1_TLS.ipynb">$\leftarrow$ previous notebook </a><br>
        <a href="py_exercise_3_1_TLS.ipynb" style="font-size:13px">III.1 Population Inversion in a Two-Level-System using Krotov's Method</a>
    </th>
    <td style="width:33%; text-align:center; font-size:16px">
        <a href="py_exercise_2_2_lambda.ipynb">$\uparrow$ previous part $\uparrow$</a><br>
        <a href="py_exercise_2_2_lambda.ipynb" style="font-size:13px">II.2 Parameter Optimization for STIRAP</a>
    </td>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
        <a href="py_exercise_3_3_chiral.ipynb">next notebook $\rightarrow$</a><br>
        <a href="py_exercise_3_3_chiral.ipynb" style="font-size:13px">III.3 Using Krotov's method to separate chiral molecules</a>
    </th>
  </tr>
  <tr style="width: 100%">
    <td style="width:33%; text-align:center; font-size:16px">
    </td>
  </tr>
</table>

<div style="text-align: right;font-size: 16px"><a href="../Julia/jl_exercise_3_2_lambda.ipynb">👉 Julia version</a></div>

---
<!-- Autoheader end -->

# Optimal Control for STIRAP

$\newcommand{tr}[0]{\operatorname{tr}}
\newcommand{diag}[0]{\operatorname{diag}}
\newcommand{abs}[0]{\operatorname{abs}}
\newcommand{pop}[0]{\operatorname{pop}}
\newcommand{aux}[0]{\text{aux}}
\newcommand{opt}[0]{\text{opt}}
\newcommand{tgt}[0]{\text{tgt}}
\newcommand{init}[0]{\text{init}}
\newcommand{lab}[0]{\text{lab}}
\newcommand{rwa}[0]{\text{rwa}}
\newcommand{bra}[1]{\langle#1\vert}
\newcommand{ket}[1]{\vert#1\rangle}
\newcommand{Bra}[1]{\left\langle#1\right\vert}
\newcommand{Ket}[1]{\left\vert#1\right\rangle}
\newcommand{Braket}[2]{\left\langle #1\vphantom{#2} \mid #2\vphantom{#1}\right\rangle}
\newcommand{Ketbra}[2]{\left\vert#1\vphantom{#2} \right\rangle \hspace{-0.2em} \left\langle #2\vphantom{#1}\right\vert}
\newcommand{e}[1]{\mathrm{e}^{#1}}
\newcommand{op}[1]{\hat{#1}}
\newcommand{Op}[1]{\hat{#1}}
\newcommand{dd}[0]{\,\text{d}}
\newcommand{Liouville}[0]{\mathcal{L}}
\newcommand{DynMap}[0]{\mathcal{E}}
\newcommand{identity}[0]{\mathbf{1}}
\newcommand{Norm}[1]{\lVert#1\rVert}
\newcommand{Abs}[1]{\left\vert#1\right\vert}
\newcommand{avg}[1]{\langle#1\rangle}
\newcommand{Avg}[1]{\left\langle#1\right\rangle}
\newcommand{AbsSq}[1]{\left\vert#1\right\vert^2}
\newcommand{Re}[0]{\operatorname{Re}}
\newcommand{Im}[0]{\operatorname{Im}}
\newcommand{toP}[0]{\omega_{12}}
\newcommand{toS}[0]{\omega_{23}}
\newcommand{oft}[0]{\left(t\right)}
$

In this notebook, you will learn how to optimize the STIRAP protocol for the lambda system we have studied already in [Exercise I.2](py_exercise_1_2_lambda.ipynb) and [Exercise II.2](py_exercise_2_2_lambda.ipynb) using the [`krotov` Python package](https://github.com/qucontrol/krotov), similarly to [Exercise III.1](py_exercise_3_1_TLS.ipynb).

## Setup

In [ ]:
import qutip # QUantum Toolbox In Python
import qutip as qt # QUantum Toolbox In Python
import numpy as np # package for numerical functions such as cos, sin, etc.
import scipy
import matplotlib # package for plotting
import matplotlib.pylab as plt

import krotov # package for optimizing with krotov

from utils.exercise_3_lambda import *

## Model

Our model consists of the "Lambda system" shown below which we already discussed
in the respective notebooks in part 1 and part 2.
As a reminder, the Hamiltonian in the rotating frame is given by:

\begin{align*}
  H = \begin{pmatrix}
        0                    & \frac{1}{2}\Omega_{P}^{*}(t) & 0 \\
    \frac{1}{2}\Omega_{P}(t) & \Delta_P                     & \frac{1}{2}\Omega_{S}^{*}(t)\\
        0                    & \frac{1}{2}\Omega_{S}(t)     & \Delta_P-\Delta_S
  \end{pmatrix}
\end{align*}

where we use the same definitions as in [Exercise I.2](py_exercise_1_2_lambda.ipynb), i.e.

$\Delta_{\mathrm{P}} = \left(\omega_2 - \omega_1\right) - \omega_{\mathrm{P}}$ and
$\Delta_{\mathrm{S}} = \left(\omega_2 - \omega_3 \right) - \omega_{\mathrm{S}}$.

Note that the Rabi frequencies are complex, i.e.

$\Omega_{\mathrm{P}} = \Omega^{(1)}_{\mathrm{P}} + i\Omega^{(2)}_{\mathrm{P}}$ and
$\Omega_{\mathrm{S}} = \Omega^{(1)}_{\mathrm{S}} + i\Omega^{(2)}_{\mathrm{S}}$.

In the following, we will optimize the real and imaginary part of
$\Omega_{\mathrm{S}}$ and $\Omega_{\mathrm{P}}$ independently.

<center><img src="../figures/lambda_system_levels.png" alt="Lambda system considered in this notebook" width="500"></center>

First, we set up the Hamiltonian using QuTiP.

In [ ]:
# Parameters
ω1 = 0.0
ω2 = 10.0
ω3 = 5.0
ω_P = 9.5  # 10.#
ω_S = 4.5  #  5.#

In [ ]:
Δ_P = (ω2 - ω1) - ω_P
Δ_S = (ω2 - ω3) - ω_S

We begin with the drift Hamltonian:

In [ ]:
H0 = qt.Qobj(np.diag([0, Δ_P, Δ_P - Δ_S]))

Next, we define the "pump" Hamiltonian:

In [ ]:
H1P_re = qt.Qobj([[0, 1, 0], [1, 0, 0], [0, 0, 0]]) / 2
H1P_im = qt.Qobj([[0, 1j, 0], [-1j, 0, 0], [0, 0, 0]]) / 2

And lastly, the "Stokes" Hamiltonian:

In [ ]:
H1S_re = qt.Qobj([[0, 0, 0], [0, 0, 1], [0, 1, 0]]) / 2
H1S_im = qt.Qobj([[0, 0, 0], [0, 0, 1j], [0, -1j, 0]]) / 2

The final part of the Hamiltonian's setup is their combination to the total Hamiltonian (cf. ['Using Krotov with
QuTiP'](https://qucontrol.github.io/krotov/v1.2.1/08_qutip_usage.html)). We
leave the control field undefined for now (`None`). This will be amended in
the following section.

In [ ]:
H = [H0, [H1P_re, None], [H1P_im, None], [H1S_re, None], [H1S_im, None]]

## Guess pulses

We choose an initial guess consisting of two low-intensity Blackman pulses.
Initially, these are chosen to be real-valued. That is, the guess for the control Hamiltonians governing the imaginary part of the Rabi frequencies will be zero. We define two constructors that return
functions with the required interface.

In [ ]:
def shape_field_real(t_start, t_stop, eps0):
    def field_shaped(t, args):
        return eps0 * krotov.shapes.blackman(t, t_start, t_stop)

    return field_shaped

In [ ]:
def shape_field_imag():
    def field_shaped(t, args):
        return 0

    return field_shaped

We can use these to update the pulses in the Hamiltonian

In [ ]:
H[1][1] = shape_field_real(t_start=1, t_stop=5, eps0=1)
H[2][1] = shape_field_imag()
H[3][1] = shape_field_real(t_start=0, t_stop=4, eps0=1)
H[4][1] = shape_field_imag()

These pulses are defined on the following time grid

In [ ]:
tlist = np.linspace(0.0, 5, 501)

and look as follows

In [ ]:
def plot_pulse(pulse, tlist, ax, title):
    if callable(pulse):
        pulse = np.array([pulse(t, args=None) for t in tlist])
    ax.plot(tlist, pulse)
    ax.set_xlabel('time')
    ax.set_ylabel('pulse amplitude')
    ax.set_title(title)

In [ ]:
fig, ax = plt.subplots(2, 2)
plot_pulse(H[1][1], tlist, ax[0, 0], title='$\\mathrm{Re}[\\Omega_P]$')
plot_pulse(H[2][1], tlist, ax[0, 1], title='$\\mathrm{Im}[\\Omega_P]$')
plot_pulse(H[3][1], tlist, ax[1, 0], title='$\\mathrm{Re}[\\Omega_S]$')
plot_pulse(H[4][1], tlist, ax[1, 1], title='$\\mathrm{Im}[\\Omega_S]$')
plt.tight_layout()
plt.show(fig)

After having set up everything, let's see how good our guess is!
To that end, we simulate the dynamics of the initial state

In [ ]:
ket1 = qt.basis(3, 0)

To keep track of the populations, we define the following projectors:

In [ ]:
proj1 = qt.projection(3, 0, 0)
proj2 = qt.projection(3, 1, 1)
proj3 = qt.projection(3, 2, 2)

In [ ]:
guess_dynamics = qutip.mesolve(H, ket1, tlist, e_ops=[proj1, proj2, proj3])

In [ ]:
fig, ax = plt.subplots()
ax.plot(
    guess_dynamics.times,
    guess_dynamics.expect[0],
    label=r'$|\langle 1|\psi|1\rangle|^2$',
)
ax.plot(
    guess_dynamics.times,
    guess_dynamics.expect[1],
    label=r'$|\langle 2|\psi|2\rangle|^2$',
)
ax.plot(
    guess_dynamics.times,
    guess_dynamics.expect[2],
    label=r'$|\langle 3|\psi|3\rangle|^2$',
)
ax.legend()
ax.set_xlabel('time')
ax.set_ylabel('population')
plt.show(fig)

## Problem 1: Objective

As already mentioned in the notebook for [Exercise III.1](py_exercise_3_1_TLS.ipynb), the
[`optimize_pulse`](https://krotov.readthedocs.io/en/stable/API/krotov.optimize.html#krotov.optimize.optimize_pulses)
method takes so-called "objectives". These contain the information about the
quantum states that span the optimization, their dynamics, and the "target" of
those dynamics.

Try to define the objective according to our optimization goal of transferring
$\ket{1}$ to $\ket{3}$

In [ ]:
ket3 = qt.basis(3, 2)

In [ ]:
objective = '---'

In [ ]:
#problem_1.hint
#problem_1.solution

## Problem 2: Specifying pulse options

Now that our Hamiltonian is completely set up and the objective for our
optimization is clear, we need to specify some parameters for Krotov's
method, specifically, a dictionary of "pulse options".

First, we define a "pulse shape". This must be a function in the range [0, 1]
which switch on from zero at the beginning of the time grid, and goes back down
to zero again at the end of the time grid. It furthermore scales the pulse update in
Krotov's method at each point in time, and thus ensures that the physical
boundary conditions of the control fields are maintained (i.e. they must remain
zero at the beginning and end of the time grid). We begin with the following choice

In [ ]:
def update_shape(t):
    """Scales the Krotov methods update of the pulse value at the time t"""
    return krotov.shapes.flattop(t, 0.0, 5.0, t_rise=0.0001, func='sinsq')

**a)** Experiment with the `t_rise` parameter and plot the update shape with
the following cell. Choose a reasonable value.

Feel free to return to this point later on to experiment with the pulse shape and its impact on the
optimization result.

In [ ]:
fig, ax = plt.subplots()
ax.plot(tlist, np.vectorize(update_shape)(tlist))
ax.set_xlabel('time')
ax.set_ylabel('update shape')
plt.show(fig)

In [ ]:
#problem_2a.solution

In addition to the "update shape", the pulse options must also include
a parameter $\lambda_a$ which determines the overall magnitude of the pulse
update in each iteration. In Krotov's update equation, the update at each
point in time is scaled with an overall factor of $S(t)/\lambda_a$, so larger
values of update suppress the update, while smaller values increase the
update, but tend to make the update more "spiky" and may lead to unphysical
results. Here, we start with a relatively safe (i.e. large) value of $\lambda_a =
100$.

**b)** After setting up the optimization in problem 3, return and choose a
better value $\lambda_a$ that yields faster convergence but does not produce
unphysical pulses.

In [ ]:
lambda_a = 100
pulse_options = {
    H[1][1]: dict(lambda_a=lambda_a, update_shape=update_shape),
    H[2][1]: dict(lambda_a=lambda_a, update_shape=update_shape),
    H[3][1]: dict(lambda_a=lambda_a, update_shape=update_shape),
    H[4][1]: dict(lambda_a=lambda_a, update_shape=update_shape)
}

In [ ]:
#problem_2b.hint
#problem_2b.solution

## Problem 3: Optimization with Krotov's method

Finally, we can use the `krotov` package's `optimize_pulses` with all the
information we build up in the previous examples.

Fill in the following missing values, which are indicated by
`'###############'` below. Proceed as follows:

**a)** Recall the API of the `optimize_pulses` function by looking at the [documentation](https://krotov.readthedocs.io/en/stable/API/krotov.optimize.html#krotov.optimize.optimize_pulses).

**b)** Which functional (and therefore which `chi_constructor`) do we need here?

Check the corresponding section in the [documentation](https://qucontrol.github.io/krotov/v1.0.0/07_krotovs_method.html#optimization-functional)
and choose from the [functionals
module](https://krotov.readthedocs.io/en/stable/API/krotov.functionals.html).

**c)** What do the values for the `check_convergence` and `iter_stop`
argument mean? Make a reasonable choice here.


**d)** Maybe your optimization is slow to converge. Adjust the relevant
parameters to obtain a better convergence (and thus better results in less time).

Ensure that the changes you apply lead to reasonable results. If one wants to optimize for an experiment, the optimized pulses need to be physical such that they can be implemented in practice.


In [ ]:
oct_result = krotov.optimize_pulses(
    '#######a#######',
    '#######a#######',
    '#######a#######',
    propagator=krotov.propagators.expm,
    #
    chi_constructor='#######b#######',
    #
    info_hook=krotov.info_hooks.print_table(
        J_T='#######b#######',
        unicode=True,
    ),
    check_convergence=krotov.convergence.Or(
        krotov.convergence.value_below('#######c#######', name='J_T'),
        krotov.convergence.delta_below('#######c#######'),
        krotov.convergence.check_motonic_error,
    ),
    iter_stop='#######c#######',
)

In [ ]:
#problem_3b.hint

In [ ]:
#problem_3c.hint

In [ ]:
#problem_3d.hint

In [ ]:
#problem_3.solution

In [ ]:
oct_result

## Problem 4: Analyzing the results

Finally, we can have a look at our solution.

**a)** Obtain the resulting objectives from the
[oct_result](https://krotov.readthedocs.io/en/stable/API/krotov.result.html)
and use mesolve to simulate the dynamics under the optimized pulse (as you did in
3d)).

In [ ]:
opt_dynamics = '-------'

In [ ]:
#problem_4a.hint
#problem_4a.solution

After simulating the optimized dynamics we can plot them via

In [ ]:
fig, ax = plt.subplots()
ax.plot(opt_dynamics.times, opt_dynamics.expect[0], label='Projector 1')
ax.plot(opt_dynamics.times, opt_dynamics.expect[1], label='Projector 2')
ax.plot(opt_dynamics.times, opt_dynamics.expect[2], label='Projector 3')
ax.legend()
ax.set_xlabel('time')
ax.set_ylabel('population')
plt.show(fig)

Now we can also extract the optimized pulses and plot the amplitudes and
phases of the pulses. To do this, you can use the following function, which
takes the real and the imaginary part of the pulse and plot the amplitude and
the phase:

In [ ]:
def plot_pulse_amplitude_and_phase(pulse_real, pulse_imaginary, tlist):
    ax1 = plt.subplot(211)
    ax2 = plt.subplot(212)
    amplitudes = [
        np.sqrt(x * x + y * y) for x, y in zip(pulse_real, pulse_imaginary)
    ]
    phases = [
        np.arctan2(y, x) / np.pi for x, y in zip(pulse_real, pulse_imaginary)
    ]
    ax1.plot(tlist, amplitudes)
    ax1.set_xlabel('time')
    ax1.set_ylabel('pulse amplitude')
    ax2.plot(tlist, phases)
    ax2.set_xlabel('time')
    ax2.set_ylabel('pulse phase (π)')
    plt.show()

**b)** Plot the optimized controls, which are contained in the
[oct_result](https://krotov.readthedocs.io/en/stable/API/krotov.result.html).

In [ ]:
print("pump pulse amplitude and phase:")
plot_pulse_amplitude_and_phase(
        "--real-pump-controls--",
        "--imag-pump-controls--",
        tlist
    )

In [ ]:
print("Stokes pulse amplitude and phase:")
plot_pulse_amplitude_and_phase(
        "--real-stokes-controls--",
        "--imag-stokes-controls--",
        tlist
    )

In [ ]:
#problem_4b.solution

## Next steps

If you are interested in more examples of gradient-based optimization, have a look at [Exercise III.3](py_exercise_3_3_chiral.ipynb) which covers the topic of three-wave-mixing to distinguish between enantiomers of a chiral molecule. Alternatively, if you are more interested in gate optimization for quantum information applications, we recommend [Exercise III.4](py_exercise_3_4_gate.ipynb).

<!-- Autofooter begin -->

---

[⬆︎ jump to top](#navtitle_3_2_py)
<!-- Autofooter end -->